<a href="https://colab.research.google.com/github/ricotta-jpgomes/yt_datatools/blob/main/ytSearch_video_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YouTube Video Search
<br>

Esse notebook apresenta uma sequência de código para buscar vídeos a partir de um termo definido pelo usuário, retornando um conjunto de dados sobre os vídeos encontrados em um arquivo .csv

In [ ]:
!pip install --upgrade google-api-python-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.4 MB 14.7 MB/s 
  Attempting uninstall: google-auth-httplib2
    Found existing installation: google-auth-httplib2 0.0.4
    Uninstalling google-auth-httplib2-0.0.4:
      Successfully uninstalled google-auth-httplib2-0.0.4
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.11
    Uninstalling google-api-python-client-1.12.11:
      Successfully uninstalled google-api-python-client-1.12.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.326 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 2.64.0 which is incompatible.


In [ ]:
! pip install python_dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from googleapiclient.discovery import build
from datetime import datetime, timedelta, timezone, date
from dotenv import load_dotenv # bibilioteca para ler minha api_key salva como uma variável de ambiente.

import os 
import pandas as pd

In [ ]:
# Carregando variáveis de ambiente
load_dotenv('/content/drive/MyDrive/Colab Notebooks/youTube/.env') # Tudo certo!

True

In [ ]:
API_KEY = os.environ['api_key']

In [ ]:
youtube = build('youtube','v3', developerKey=API_KEY)

In [ ]:
query = input()
nextPage_token = None

In [ ]:
videos = []

while True:
  search_response = youtube.search().list(
    q=query, 
    part='snippet', 
    type='video',
    maxResults = 50,
    pageToken = nextPage_token
    ).execute()

  for search_result in search_response.get('items'):
      videos.append(search_result)
      
  nextPage_token = search_response.get('nextPageToken')
  if nextPage_token is None:
    break

In [ ]:
videos_ids = list(map(lambda x : x['id']['videoId'], videos))

In [ ]:
videos_stats = []

for id in videos_ids:
  res = youtube.videos().list(part='statistics', id=id).execute()
  videos_stats.append(res['items'][0])

In [ ]:
videos_titles = list(map(lambda x: x['snippet']['title'], videos))
videos_descriptions = list(map(lambda x: x['snippet']['description'], videos))
channels_ids = list(map(lambda x: x['snippet']['channelId'], videos))
channels_titles = list(map(lambda x: x['snippet']['channelTitle'], videos))
published_date = list(map(lambda x: str(x['snippet']['publishedAt']), videos)) 

In [ ]:
views = list(map(lambda x: int(x['statistics']['viewCount']), videos_stats))

In [ ]:
likes = []
comments = []

for stat in videos_stats:
  s = stat['statistics']

  if 'commentCount' in s:
      comments.append( int(s['commentCount']) )
  else:
      comments.append( 0 )

  if 'likeCount' in s:
      likes.append( int(s['likeCount']) )
  else:
      likes.append( 0 )

In [ ]:
diff = timedelta(hours = -3)

t_zone = timezone(diff)

extraction_date = [str(datetime.now(tz = t_zone))]*len(videos)
extraction_day = date.today()

In [ ]:
videos_df = pd.DataFrame({
    'id' : videos_ids,
    'title' : videos_titles,
    'description' : videos_descriptions,
    'publishedAt' : published_date,
    'extractedAt' : extraction_date,
    'channelId' : channels_ids,
    'channelTitle' :channels_titles,
    'views' : views,
    'likes' : likes,
    'comments' : comments
    })

file_name = input('Insira aqui o nome do arquivo: ') + f'_{extraction_day}'
videos_df.to_csv(f'/content/drive/MyDrive/Datasets/{file_name}.csv', index = False)

Insira aqui o nome do arquivo: maconaria
